In [2]:
#Data Ingestion
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('inputData.pdf')
docs=loader.load()

In [3]:
docs

[Document(page_content='CONTOSO LTD.  \nContoso Headquarters  \n123 456th St \nNew York, NY , 10001  \nMicrosoft Corp  \n123 Other St,  \nRedmond WA, 98052  INVOICE  \nINVOICE : INV-100 \nINVOICE DATE : 11/15/20 19 \nDUE DATE:  12/15/20 19 \nCUSTOMER NAME:  MICROSOFT CORPORATION  \nSERVICE PERIO D: 10/14/2019 – 11/14/2019  \nCUSTOMER ID:  CID-12345  \nBILL TO: \nMicrosoft Finance  \n123 Bill St,  \nRedmond WA, 98052  SHIP TO:  \nMicrosoft Delivery  \n123 Ship St,  \nRedmond WA, 98052  SERVICE ADDRESS:  \nMicrosoft Services  \n123 Service St,  \nRedmond WA, 98052  \nSALESPERSON  P.O. NUMBER  REQUISITIONER  SHIPPED VIA  F.O.B. POINT  TERMS  \nPO-3333  \nDATE  ITEM CODE  DESCRIPTION  QTY  UM PRICE  TAX  AMOUNT  \n3/4/20 21 A123 Consulting Services  2 hours $30.00  $6.00  $60.00  \n3/5/2021  B456 Document Fee 3 $10.00  $3.00 $30.00  \n3/6/2021  C789 Printing Fee  10 pages  $1.00  $1.00  $10.00  \nSUBTOTAL  $100.00  \nSALES TAX  $10.00  \nTOTAL  $110.00  \nPREVIOUS UNPAID BALANCE  $500.00  

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=text_splitter.split_documents(docs)
documents

[Document(page_content='CONTOSO LTD.  \nContoso Headquarters  \n123 456th St \nNew York, NY , 10001  \nMicrosoft Corp  \n123 Other St,  \nRedmond WA, 98052  INVOICE  \nINVOICE : INV-100 \nINVOICE DATE : 11/15/20 19 \nDUE DATE:  12/15/20 19 \nCUSTOMER NAME:  MICROSOFT CORPORATION  \nSERVICE PERIO D: 10/14/2019 – 11/14/2019  \nCUSTOMER ID:  CID-12345  \nBILL TO: \nMicrosoft Finance  \n123 Bill St,  \nRedmond WA, 98052  SHIP TO:  \nMicrosoft Delivery  \n123 Ship St,  \nRedmond WA, 98052  SERVICE ADDRESS:  \nMicrosoft Services', metadata={'source': 'inputData.pdf', 'page': 0}),
 Document(page_content='Microsoft Delivery  \n123 Ship St,  \nRedmond WA, 98052  SERVICE ADDRESS:  \nMicrosoft Services  \n123 Service St,  \nRedmond WA, 98052  \nSALESPERSON  P.O. NUMBER  REQUISITIONER  SHIPPED VIA  F.O.B. POINT  TERMS  \nPO-3333  \nDATE  ITEM CODE  DESCRIPTION  QTY  UM PRICE  TAX  AMOUNT  \n3/4/20 21 A123 Consulting Services  2 hours $30.00  $6.00  $60.00  \n3/5/2021  B456 Document Fee 3 $10.00  $

In [15]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
db=Chroma.from_documents(documents,OllamaEmbeddings())

In [11]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

In [14]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(""" Analyze the following text and provide a accurate answer.
                                        <context>{context}</context>
                                        Question: {input}"""

                                    )

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm, prompt)

In [17]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000256A9DF33D0>)

In [18]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [23]:
response=retrieval_chain.invoke({"input":"Fetch me all Date, Item code,Description,QTY,UM,Price,TAX,Amount"})

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [28]:
print(response['answer'])


Based on the provided text, here are the requested values:

* Dates:
	+ 3/4/20
	+ 3/5/20
	+ 3/6/20
	+ 11/15/19
	+ 12/15/19
	+ 10/14/2019
	+ 11/14/2019
* Item Codes:
	+ A123
	+ B456
	+ C789
* Descriptions:
	+ Consulting Services
	+ Document Fee
	+ Printing Fee
* Quantities (QTY):
	+ 2 hours
	+ 3
	+ 10 pages
* Units of Measure (UM):
	+ $/hour
	+ $/each
	+ $/page
* Prices:
	+ $30.00
	+ $10.00
	+ $1.00
* Taxes:
	+ $6.00
	+ $3.00
	+ $1.00
* Amounts:
	+ $60.00
	+ $30.00
	+ $10.00

Please note that there are multiple instances of each item, but I have only listed the unique values for clarity purposes.
